# Sentiment prediction
## UCI Drug Dataset

Load in required libraries 

In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from sklearn.metrics import roc_auc_score
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
import multiprocessing as mp
import csv

In [ ]:
# Load Data
df_train = pd.read_csv(
    './project2_data/drugsCom_raw/drugsComTrain_raw.tsv', delimiter='\t', encoding='utf-8')
df_test = pd.read_csv(
    './project2_data/drugsCom_raw/drugsComTest_raw.tsv', delimiter='\t', encoding='utf-8')

In [ ]:
###########################################
#Word Cloud
%matplotlib inline
pos_tweets = df_train[df_train.rating >= 5]
pos_string = []
for t in pos_tweets.review:
    pos_string.append(t)
pos_string = pd.Series(pos_string).str.cat(sep=' ')

wordcloud = WordCloud(width=1600, height=800, max_font_size=200).generate(pos_string)
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('Positive reviews')
plt.axis("off")
plt.show()

neg_tweets = df_train[df_train.rating < 5]
neg_string = []
for t in neg_tweets.review:
    neg_string.append(t)
neg_string = pd.Series(neg_string).str.cat(sep=' ')

wordcloud = WordCloud(width=1600, height=800, max_font_size=200).generate(neg_string)
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('Negative reviews')
plt.axis("off")
plt.show()

In [ ]:
#Pre process the comments and generate word2vec embeddings
# process_chunk for converting a given comment into a word embedding for later use
def process_chunk(comments):
    processed_comments = []
    lookup = {}
    for comment in comments:
        idxs = []
        for i, word in enumerate(comment.split(' ')):
            # if the comment is longer than the max-seq-length break the  loop
            if i > max_seq_length:
                break

            # put everything into lowercase
            word = word.lower()

            # if empty word skip
            if word == '':
                continue
            # if word is in our lookup table
            if word in lookup.keys():
                idx = lookup[word]
            else:
                try:
                    idx = glove_word[glove_word == word].index[0]
                except IndexError:
                    idx = vocab_size
                # put word into lookup table
                lookup[word] = idx
            idxs.append(idx)
        processed_comments.append(idxs)

    processed_comments = keras.preprocessing.sequence.pad_sequences(
        processed_comments,
        maxlen=max_seq_length,
        padding='post',
        truncating='post',
        value=0
    )
    return processed_comments

# Pre process the dataset; Remove punctuation marks etc. 
def process_comments(comments):
    pre_comments = []
    for comment in comments:
        comment = comment.replace(',', '')
        comment = comment.replace('.', '')
        comment = comment.replace('(', '')
        comment = comment.replace(')', '')
        comment = comment.replace('"', '')
        comment = comment.replace('&amp;', 'and')
        comment = comment.replace('!', '')
        comment = comment.replace('/', ' ')
        comment = comment.replace('&quot;', ' ')
        comment = comment.replace('&#39;', ' ')
        comment = comment.replace('\n', ' ')
        comment = comment.replace('\r', ' ')
        pre_comments.append(comment)
    return np.array(pre_comments)

# Parallelize the word embedding process 
def comments_to_idxs(comments):
    cores = mp.cpu_count()
    pool = mp.Pool(cores)
    chunks = np.array_split(comments, cores)
    results = pool.map(process_chunk, chunks)
    results = np.vstack(results)
    return results

# The tf-idf features are extracted as a first resort for the model
def getCommentFeatures(review,df,n_features=1000):
    tvec = TfidfVectorizer(max_features=n_features)
    X = tvec.fit_transform(review).todense().A
    return [X,tvec]

# Discretize the ratings into three classes as specified
def getDiscreteLabels(ratings):
    temp_ratings = ratings.copy()
    for i, curr_rating in enumerate(ratings):
        if (curr_rating >= 7):
            temp_ratings[i] = 1
        elif (curr_rating >= 4):
            temp_ratings[i] = 0
        else:
            temp_ratings[i] = -1

    return temp_ratings

#Training performed on undersampled data: 
#Specifically undersample the positive class becuase it is at least 4 times as big 
def undersample_data(y_in):

    np.random.seed(0)
    n_samples = y_in.shape[0]
    idxs_inclusion = [False]*n_samples
    for i in range(n_samples):
        if(y_in[i] == 1):
            if(np.random.choice(np.arange(5),size=1)==0):
                idxs_inclusion[i] = True
        else:
            idxs_inclusion[i] = True

    return np.array(idxs_inclusion)


In [ ]:
review_train = process_comments(df_train.review.values)
review_test = process_comments(df_test.review.values)

### Undersample the data
We first undersampled the data. Specifically the positive class is under sampled to imrpove trainng times. 

In [ ]:
[X,tvec] = getCommentFeatures(review_train,df_train,n_features=1000) # Entire training data
y = getDiscreteLabels(ratings = df_train['rating'].values) # Entire test data

X_test_data = tvec.transform(review_test)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0) # Split for validation

%matplotlib inline
plt.hist(y_train,align = 'mid')
plt.title('Class distribution before undersampling')
plt.show()

In [ ]:
#Undersample the training data
idxs_inclusion = undersample_data(y_train)
X_train = X_train[idxs_inclusion,:]
y_train = y_train[idxs_inclusion]

%matplotlib inline
plt.hist(y_train,align = 'mid')
plt.title('Class distribution after undersampling')
plt.show()

### Try different models as a baseline approach 
We tried the RandomForests and Adaboost methods as part of ensemble methods. 
We also used naive bayes and logistic regression models. 
SVM was not used in the inteerst of training times. 

In [ ]:
def getScores(model, model_name, metrics):
    print("Model: ", model_name)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average="weighted")
    acc = accuracy_score(y_test, y_pred)

    metrics = metrics.append({"Name": model_name, "F1_score": f1, "Accuracy": acc}, ignore_index=True)

    return [y_pred, metrics]

metrics = pd.DataFrame(data=[], columns=["Name", "F1_score", "Accuracy"])

rfc = RandomForestClassifier(class_weight='balanced', n_estimators=50)
lg = LogisticRegression(class_weight='balanced')
adb = ensemble.AdaBoostClassifier()
cb = naive_bayes.ComplementNB()

model_names = ["RandomForest", "LogisticRegression", "Adaboost", "ComplementNB"]
models = [rfc, lg, adb, cb]

all_models = zip(models, model_names)
for model, model_name in all_models:
    _, metrics = getScores(model, model_name, metrics)

print(metrics)

### DL Model 

In [ ]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from os import environ as cuda_environment

cuda_environment["CUDA_VISIBLE_DEVICES"] = str(0)

input_shape = X.shape[1:]

model = Sequential()
model.add(Dense(500, input_shape=input_shape, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(10, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

def getCategorical(y):
    n_samples = y.shape[0]
    y_one_hot = np.zeros((n_samples, 3))
    for i in range(y.shape[0]):
        if (y[i] == 1):
            y_one_hot[i, 0] = 1
        elif (y[i] == 0):
            y_one_hot[i, 1] = 1
        else:
            y_one_hot[i, 2] = 1

    return y_one_hot

def flatten(y_to_flatten):
    ytemp = np.argmax(y_to_flatten, axis=1)
    y_pred_fl = ytemp.copy()
    y_pred_fl[ytemp == 2] = -1
    y_pred_fl[ytemp == 0] = 1
    y_pred_fl[ytemp == 1] = 0

    return y_pred_fl


In [ ]:
cat_y_train = getCategorical(y_train)
cat_y_test = getCategorical(y_test)

history = model.fit(X_train, cat_y_train, epochs=10, validation_data=(X_test, cat_y_test))

%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

cat_y_pred = model.predict(X_test)
y_pred = flatten(cat_y_pred)

f1 = f1_score(y_test, y_pred, average="weighted")
acc = accuracy_score(y_test, y_pred)
metrics = metrics.append({"Name": "ANN", "F1_score": f1, "Accuracy": acc}, ignore_index=True)

print(metrics)

### Word embedding 

In [ ]:
# ! wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
# ! mkdir -p project2_data/glove
# ! unzip -d project2_data/glove/ glove.twitter.27B.zip
vocab_size = 10000
vector_length = 25
max_seq_length = 500
glove_embeddings = pd.read_csv('project2_data/glove/glove.twitter.27B.25d.txt',
                               header=None, delimiter=' ', encoding='utf-8', quoting=csv.QUOTE_NONE)
print('Loaded glove embeddings')
glove_embeddings = glove_embeddings[:vocab_size]

glove_word = glove_embeddings[0]

glove_vectors = glove_embeddings.iloc[:, 1:].values
M1 = np.zeros((vocab_size + 1, vector_length + 1))
M1[:-1, :-1] = glove_vectors
glove_vectors = M1
glove_vectors[-1, -1] = 1


In [ ]:
# Some Drug-Names appear multiple time in a non-unique fashion.
# e.g. "Stalevo", "Stalevo 200"
drug_names_raw = df_train.drugName.unique()

for idx,drug in enumerate(drug_names_raw):
    drug = drug.lower()
    drug_names_raw[idx] = drug

drug_names = np.copy(drug_names_raw)
for idx1, drug1 in enumerate(drug_names_raw):
    for idx2, drug2 in enumerate(drug_names_raw):
        if drug1 in drug2 and drug1 != drug2 and drug2 in drug_names:
            idx_ = np.where(drug_names == drug2)
            drug_names = np.delete(drug_names, idx_)

print(f'Deleted {len(drug_names_raw) - len(drug_names)} duplicate drug_names')


In [ ]:
##################################
#Martin code
# Find the mean rating of all drugs:
# Split 
df_train_model, df_valid = train_test_split(df_train, random_state=42)

mean_rating = np.mean(df_train_model.rating)

# Find mean Rating for each Drugname
drug_ratings = {}
for drug_name in drug_names:
    idxs = [drug_name in name for name in df_train_model.drugName.values]
    ratings = df_train_model.rating[idxs]
    mean_rating = np.mean(ratings)
    drug_ratings[drug_name] = mean_rating
print('Found the mean ratings for each drug')

# Find ratings for df_valid
valid_ratings = []
num_not_found = 0
for valid_drug in df_valid.drugName:
    ratings = []
    # The drug listing could contain multiple drugs
    # e.g. 'drug A, drug B'
    # we take the average rating of each drug
    for key in drug_ratings.keys():
        if key in valid_drug:
            ratings.append(drug_ratings[key])

    # If no drug with that name is found then the rating is set to the mean rating
    if ratings == []:
        ratings.append(mean_rating)
        num_not_found += 1
    rating = np.mean(ratings)
    valid_ratings.append(rating)
print(f'Number of not found drugs {num_not_found}')
print(f'MSE: {mean_squared_error(df_valid.rating.values, valid_ratings)}')

def to_labels(ratings):
    labels = []
    for rating in ratings:
        if rating > 7:
            label = 0
        elif rating < 4:
            label = 2
        else:
            label = 1
        labels.append(label)
    return labels

valid_true_labels = to_labels(df_valid.rating.values)
valid_pred_labels = to_labels(valid_ratings)
print(f'Accuracy: {accuracy_score(valid_true_labels, valid_pred_labels):.3} - f1: {f1_score(valid_true_labels, valid_pred_labels, average="micro"):.3}')

In [ ]:
####################

train_idxs = process_chunk(review_train)
test_idxs = process_chunk(review_test)

np.save('project2_data/glove/train_idxs.npy', train_idxs)
np.save('project2_data/glove/test_idxs.npy', test_idxs)

#train_idxs = np.load('project2_data/glove/train_idxs.npy')
#test_idxs = np.load('project2_data/glove/test_idxs.npy')

max_seq_length = 50
train_idxs = train_idxs[:, :max_seq_length] # Entire training data
test_idxs = test_idxs[:, :max_seq_length] # Entire test data 

train_idx, valid_idx, train_y, valid_y = train_test_split(
    train_idxs, df_train.rating.values, random_state=42)
train_y_cat = pd.get_dummies(getDiscreteLabels(train_y)).values
valid_y_cat = pd.get_dummies(getDiscreteLabels(valid_y)).values

# Bidiretional LSTM

batch_size = 64
hidden_units = 256
epochs = 20

sequence_input = keras.layers.Input(shape=(max_seq_length,))
embedding_sequence = keras.layers.Embedding(
    vocab_size + 1,
    vector_length + 1,
    weights=[glove_vectors],
    input_length=max_seq_length,
    trainable=False
)(sequence_input)

lstm = keras.layers.Bidirectional(
    keras.layers.LSTM(hidden_units))(embedding_sequence)
drop = keras.layers.Dropout(0.5)(lstm)
dense = keras.layers.Dense(128)(drop)
out = keras.layers.Dense(3, activation='softmax')(dense)
model = keras.models.Model(inputs=sequence_input, outputs=out)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model.fit(x = train_idx,y = train_y_cat,validation_data=(valid_idx, valid_y_cat),epochs=epochs)

model.save('model_cat.h5')


#model = keras.models.load_model('model_cat.h5')
#print('loaded model')

lstm_predictions = model.predict(valid_idx)
print(f'ROC AUC {roc_auc_score(valid_y_cat, lstm_predictions):.3}  Acc: {accuracy_score(np.argmax(valid_y_cat, axis=1), np.argmax(lstm_predictions, axis=1)):.3} ')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

combined = np.hstack([pd.get_dummies(to_labels(valid_ratings)).values, lstm_predictions])

train_combined, valid_combined, train_cat, valid_cat = train_test_split(combined, np.argmax(valid_y_cat, axis=1))

clf = SVC()
clf.fit(train_combined, train_cat)
print(clf.score(valid_combined, valid_cat))